In [ ]:
# load_wine_into_db.ipynb
# The Wine Mart
# Application for 'Koodoo'
# Applicant: Neel Patel
# Email: neel@quarithm.com

In [2]:
import pandas as pd
import numpy as np
import sqlite3

In [17]:
# import Wine records from the CSV file
df = pd.read_csv("Wines.csv")

# How Many rows in the Staging_wines table?

24997 rows before cleaning. 23375 wines with a price

# What is the average price of a bottle of wine?
Average price: $3546.71

# How much is the most expensive wine?

$250000

# What would happen if you deleted a record from DimWinery table?
When retrieving Facts about a particular set of wines, the link between the tables would be lost. The data would be corrupted.

# How could you prevent that deletion from occurring?
A TRIGGER could be used on the SQL server that checks to see if there are any records with the same Winery_Id. If it exists then the deletion is not permitted.

# What process can be used to automate a weekly file download from an SFTP server and then update The Wine Mart from the new file?

- https://www.ftpgetter.com/ 

FTPGetter could be used to schedule downloads of one or more files and then trigger a script - either a batch file or launch a particular software from a command line interface if needed.

# What changes could be made to improve the The Wine Mart design to provide an audit trail of when different wines are added and/or updated?

An audit table could be added to the schema along with TRIGGERs that would add information of who added/deleted/updated a particular record, at what time and other relevant information.

In [18]:
df.head()

,Vintage,Country,County,Designation,Points,Price,Province,Title,Variety,Winery
0,01/01/1919,Spain,Cava,1919 Brut Selecció,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,01/01/1929,Italy,Vernaccia di San Gimignano,NaN,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,01/01/1929,Italy,Sangiovese di Romagna Superiore,Prugneto,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,01/01/1934,Portugal,NaN,Reserva Velho,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,01/01/1945,France,Rivesaltes,Legend Vintage,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24997 entries, 0 to 24996
Data columns (total 10 columns):
Vintage        24997 non-null object
Country        24989 non-null object
County         20809 non-null object
Designation    17633 non-null object
Points         24997 non-null int64
Price          23375 non-null object
Province       24989 non-null object
Title          24997 non-null object
Variety        24997 non-null object
Winery         24997 non-null object
dtypes: int64(1), object(9)
memory usage: 1.9+ MB


In [25]:
df2_copy = df.copy()

In [29]:
# removing non numeric characters
df['Price'] = df['Price'].str.replace('$','')
df['Price'] = df['Price'].str.replace(',','')
df['Price'] = df['Price'].str.replace('.','')

In [30]:
df['Price'] = pd.to_numeric(df['Price'])
df.head()

,Vintage,Country,County,Designation,Points,Price,Province,Title,Variety,Winery
0,01/01/1919,Spain,Cava,1919 Brut Selecció,88,1300,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,01/01/1929,Italy,Vernaccia di San Gimignano,NaN,87,1400,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,01/01/1929,Italy,Sangiovese di Romagna Superiore,Prugneto,84,1500,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,01/01/1934,Portugal,NaN,Reserva Velho,93,49500,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,01/01/1945,France,Rivesaltes,Legend Vintage,95,35000,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [32]:
# display states on the distribution of the price of wines
df['Price'].describe()

count     23375.000000
mean       3546.708021
std        4083.522016
min         400.000000
25%        1700.000000
50%        2500.000000
75%        4200.000000
max      250000.000000
Name: Price, dtype: float64

In [87]:

# creating seperate dataframes for each table in The Wine Mart database

DimGeography_df = pd.DataFrame([df['Country'].unique(), df['Province'].unique(), df['County'].unique()])
DimGeography_df = DimGeography_df.T
DimGeography_df.columns=['Country', 'County', 'Province']
DimGeography_df

,Country,County,Province
0,Spain,Catalonia,Cava
1,Italy,Tuscany,Vernaccia di San Gimignano
2,Portugal,Central Italy,Sangiovese di Romagna Superiore
3,France,Colares,NaN
4,US,Languedoc-Roussillon,Rivesaltes
5,Austria,Port,Sonoma Valley
6,Germany,Moscatel do Douro,Champagne
7,South Africa,California,Sonoma County
8,Hungary,Champagne,Carneros
9,Australia,Kamptal,Anderson Valley


In [82]:
DimVariety_df = pd.DataFrame([df['Variety'].unique()])
DimVariety_df = DimVariety_df.T
DimVariety_df.columns=['Variety']
DimVariety_df

,Variety
0,Sparkling Blend
1,Vernaccia
2,Sangiovese
3,Ramisco
4,Red Blend
5,Port
6,Moscatel
7,Cabernet Sauvignon
8,Champagne Blend
9,Riesling


In [88]:
DimWinery_df = pd.DataFrame([df['Winery'].unique()])
DimWinery_df = DimWinery_df.T
DimWinery_df.columns=['Winery']
DimWinery_df

,Winery
0,L'Arboc
1,Guidi 1929
2,Poderi dal Nespoli 1929
3,Adega Viuva Gomes
4,Gérard Bertrand
5,Burmester
6,Messias
7,Barros
8,Kopke
9,Adega de Favaios


In [91]:
FactWine_df = pd.DataFrame([df['Title'], df['Points'], df['Price'], df['Vintage']])
FactWine_df = FactWine_df.T
FactWine_df.columns=['Title', 'Points', 'Price', 'Vintage']
FactWine_df

,Title,Points,Price,Vintage
0,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),88,1300,01/01/1919
1,Guidi 1929 2015 Vernaccia di San Gimignano,87,1400,01/01/1929
2,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,84,1500,01/01/1929
3,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,93,49500,01/01/1934
4,Gérard Bertrand 1945 Legend Vintage Red (Rives...,95,35000,01/01/1945
5,Burmester 1952 Colheita Tawny (Port),96,41500,01/01/1952
6,Messias 1963 Colheita Tawny (Port),96,40000,01/01/1963
7,Barros 1974 Colheita Tawny (Port),93,17500,01/01/1974
8,Kopke 1978 Colheita Tawny (Port),94,10000,01/01/1978
9,Adega de Favaios 1980 Moscatel do Douro,93,13700,01/01/1980


# Load into Database

In [120]:
# connect to the database and  retrieve cleaned data
conn = sqlite3.connect("./the_wine_mart.sqlite")

c = conn.cursor() 

In [121]:
# load dataframes into respective database table
# 'if_exists' is set to 'replace' for now but would be different if its a weekly update.
DimGeography_df.to_sql('DimGeography', conn, if_exists='replace')
DimVariety_df.to_sql('DimVariety', conn, if_exists='replace')
DimWinery_df.to_sql('DimWinery', conn, if_exists='replace')
FactWine_df.to_sql('FactWine', conn, if_exists='replace')

In [122]:
# check the tables in the database
conn.execute("select name from sqlite_master where type='table';").fetchall()


[('DimGeography',), ('DimVariety',), ('DimWinery',), ('FactWine',)]

In [123]:
# retrieve the data from the tables
test1 = pd.DataFrame(conn.execute('''SELECT * FROM DimGeography''').fetchall())
test2 = pd.DataFrame(conn.execute('''SELECT * FROM DimVariety''').fetchall())
test3 = pd.DataFrame(conn.execute('''SELECT * FROM DimWinery''').fetchall())
test4 = pd.DataFrame(conn.execute('''SELECT * FROM FactWine''').fetchall())

In [124]:
test1.head()

,0,1,2,3
0,0,Spain,Catalonia,Cava
1,1,Italy,Tuscany,Vernaccia di San Gimignano
2,2,Portugal,Central Italy,Sangiovese di Romagna Superiore
3,3,France,Colares,None
4,4,US,Languedoc-Roussillon,Rivesaltes


In [125]:
test2.head()

,0,1
0,0,Sparkling Blend
1,1,Vernaccia
2,2,Sangiovese
3,3,Ramisco
4,4,Red Blend


In [126]:
test3.head()

,0,1
0,0,L'Arboc
1,1,Guidi 1929
2,2,Poderi dal Nespoli 1929
3,3,Adega Viuva Gomes
4,4,Gérard Bertrand


In [127]:
test4.head()

,0,1,2,3,4
0,0,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),88,1300.0,01/01/1919
1,1,Guidi 1929 2015 Vernaccia di San Gimignano,87,1400.0,01/01/1929
2,2,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,84,1500.0,01/01/1929
3,3,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,93,49500.0,01/01/1934
4,4,Gérard Bertrand 1945 Legend Vintage Red (Rives...,95,35000.0,01/01/1945


In [128]:
conn.close()